In [1]:
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import re


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [2]:
from sklearn import metrics

In [3]:
import pymorphy2

In [107]:
fname = 'comments.csv'

### Структура файлов

In [108]:
df = pd.read_csv(fname, sep='\t', header = 0, names = ['grade', 'comment'])
df.head(10)

,grade,comment
0,1,На момент покупки производителем были заявлены...
1,2,Купил этот продукт и на 3день отказал сенсор. ...
2,4,Полное барахло.
3,2,В понедельник отдам в сервис по гарантии. Посм...
4,3,"Пользуюсь уже с полгода, за такие деньги более..."
5,4,Конечно сразу обращает на себя внимание компле...
6,4,Купила этот телефон в ноябре месяце. Очень нра...
7,4,"Отличная модель, пользуюсь телефоном уже 4 год..."
8,1,"Маркой Леново остался не доволен, и больше ее..."
9,4,Смартфончик просто класс. Брал его взамен моем...


In [110]:
df[df['comment'].isnull()].head()

,grade,comment
13,1,NaN
14,1,NaN
47,0,NaN
51,0,NaN
54,3,NaN


In [111]:
# удалим строки с отсутствующими значениями
print df.shape
df = df.dropna(axis=0, how='any')
print df.shape

(33740, 2)
(31337, 2)


In [112]:
df.head(5)

,grade,comment
0,1,На момент покупки производителем были заявлены...
1,2,Купил этот продукт и на 3день отказал сенсор. ...
2,4,Полное барахло.
3,2,В понедельник отдам в сервис по гарантии. Посм...
4,3,"Пользуюсь уже с полгода, за такие деньги более..."


In [113]:
# раз заметили странное значение, можно и удалить
df.drop(df.index[2], inplace=True)

In [114]:
print df.comment[8], df.grade[8]

Маркой Леново остался  не доволен, и больше ее никогда не куплю. 1


### Читаем файлы, подготавливаем тексты

In [11]:
import random

def some(x, n):
    return x.ix[random.sample(x.index, n)]

In [65]:
n1 = list(df.comment[df['grade'] == 1])
n2 = list(df.comment[df['grade'] == 2])
p3 = list(some(df.comment[df['grade'] == 3],len(n2)))
p4 = list(some(df.comment[df['grade'] == 4],len(n1)+len(n2))) #балансируем выборку

In [66]:
negative = n1+n2 # для увеличения объема обучающей выборки
                 # используем в качестве негативных градации 1(ужасно) и 2(плохо)

In [67]:
positive = p4 # идея использовать 3 и 4 в качестве положительных оказалась плохой

In [68]:
print len(negative), len(positive)

4764 4764


In [93]:
# выведем примеры
for i in range (0,6):
    print positive[i]
    print '_' * 5
    print negative[i]
    print '_' * 25

ну тут много понаписали что гааавно не то не сё да стекло хрупковато на новое стоит всего 2400 плюс замена 500  всего делов  я лично юзаю 4 месяца доволен  и связь нормальная просто любую модель смартфона уронив на пол да тем более кафель не всегда можно вообще собрать то мой лично подол в пруд и провел там неменее минуты достал отключил высушил и все работает дальше
_____
На момент покупки производителем были заявлены неплохие характеристики аппарата при низкой его стоимости (4400р.), на что я и соблазнился. В итоге смартфон быстро сдох, но потраченных денег жалко, поэтому теперь приходится бегать по ремонтам, чтобы либо сделали, либо деньги вернули.Вам это надо? К покупке не рекомендую.
_________________________
Перешел с самсунга А5(2016),тоже неплохой аппарат,но Nova как глоток свежего воздуха.Покупал у официалов,были подарки,внешний аккумулятор на 13000а/ч ,запасной шнурок и слфипалка.Доставка бесплатна.
_____
Маркой Леново остался  не доволен, и больше ее никогда не куплю.
______

In [71]:
X = negative+positive
Y = [0 for x in range(len(negative))] + [1 for x in range(len(positive))]

In [72]:
print len(X), len(Y), sum(Y)

9528 9528 4764


In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    stratify=Y, 
                                                    test_size=0.25)

### Составляем словарь

### Pymorphy2 - лемматизация

In [26]:
morph = pymorphy2.MorphAnalyzer()

In [94]:
def text_to_words(text):
    t = text
    template = re.compile(u'([A-Za-zА-Яа-яёЁ]+)') # первый этап сохранения пунктуации в словаре
    t2 = template.split(t)
    #t1 = [value for value in t2 if value != ' ']
    t1 = [morph.parse(value)[0].normal_form for value in t2 if value != ' ']
    return ' '.join(t1)

In [76]:
%%time
X_pm = [text_to_words(x.decode('utf8')) for x in X]

CPU times: user 4min 22s, sys: 3.98 s, total: 4min 26s
Wall time: 4min 24s


In [77]:
from sklearn.model_selection import train_test_split
X_train_pm, X_test_pm, y_train, y_test = train_test_split(X_pm, Y,
                                                    stratify=Y, 
                                                    test_size=0.25)

In [78]:
print X_test_pm[0]

20.10.16  купить .25.10.16  перестать работать микрофон от слово совсем .  в разговорный динамика эхо .  пробовать везде . ещё возникнуть сложность с возврат , но тут большой вопрос к магазин .  в мыть город сервис нет . покупать в подарок .  очень расстроить соня . из комментарий видно что проблема с микрофон у многий .  поэтому это заводский сырой недочёт .


### Векторизация

#### Vectirizer (pymorphy)

In [79]:
def my_tokenizer(s):
    return s.split() # второй этап сохранения пунктуации в словаре

In [80]:
count_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = my_tokenizer,    \
                             #preprocessor = pr, \
                             #stop_words = stop_words,   \
                             stop_words = None,   \
                             #max_df = .3,        \
                             #min_df = .001,        \
                             #ngram_range=(1,3),   \
                             max_features = 15000
                            ) 

In [81]:
tfidf_vectorizer = TfidfVectorizer(
                             # sublinear_tf = True, \
                             tokenizer = my_tokenizer,    \
                             ngram_range=(1,3), \
                             max_features = 15000
                            )

In [82]:
#vect = vectorizer.fit(X_pm)
vect = tfidf_vectorizer.fit(X_pm)
#vect = count_vectorizer.fit(X_pm)

train_data_features = vect.transform(X_train_pm).toarray()
test_data_features = vect.transform(X_test_pm).toarray()

print train_data_features.shape
print test_data_features.shape

(7146, 15000)
(2382, 15000)


In [95]:
#Можно вывести самые высоко- и низко- частотные слова
vocab = vect.get_feature_names()
dist = np.sum(train_data_features, axis=0)
li = zip(vocab, dist)
li.sort(key=lambda x: x[1])
for i in li[0:10] + li[-10:]:
#for i in li[0:100]:
    print i[0]

;•
. дело в
\" самсунг
перезагрузка и
quick
китайщина
состояние ,
до 8
gprs
по телефон ,
с
я
на
-
телефон
в
и
не
.
,


In [96]:
# test
TRAIN = train_data_features
cv = 6
print cross_val_score(SGDClassifier(), TRAIN, y_train, scoring='accuracy', cv = cv).mean()
print cross_val_score(LogisticRegression(), TRAIN, y_train, scoring='accuracy', cv = cv).mean()
print cross_val_score(MultinomialNB(), TRAIN, y_train, scoring='accuracy', cv = cv).mean()
print cross_val_score(LinearSVC(), TRAIN, y_train, scoring='accuracy', cv = cv).mean()

0.85362500705
0.862574962871
0.864538567105
0.867615569718


#### Ответ для pymorphy

In [85]:
# для отложенной выборки:
regressor = MultinomialNB()
#regressor = LinearSVC()
#regressor = LogisticRegression()#(class_weight='balanced')
u = regressor.fit(train_data_features,y_train)
z = regressor.predict(test_data_features)

metrics.accuracy_score(y_test, z)

0.88455079764903444

### Результат

In [87]:
reviews = pd.read_csv('test1.csv', sep='\t', header = 0, names =  ['id','r'])

In [88]:
result = list(reviews['r'])

In [90]:
res_pm = [text_to_words(x.decode('utf8')) for x in result]

In [91]:
result_data_features = vect.transform(res_pm).toarray()
print result_data_features.shape
r5 = regressor.predict(result_data_features)

(100, 15000)


In [92]:
r5

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1])

In [97]:
columns = ['comment', 'result']
diff_df = pd.DataFrame(result) 

In [98]:
diff_df['result'] = r5

In [100]:
# подстраиваемся под формат ответа
diff_df['res2'] = diff_df['result'] 
diff_df.loc[diff_df.result ==0,['res2']] = 'neg'
diff_df.loc[diff_df.result ==1,['res2']] = 'pos'

In [115]:
res_df = pd.DataFrame(diff_df,columns = ['res2'])
res_df = df.rename(index=str, columns={"res2": "y"})
res_df.to_csv('answer4.csv', index_label='Id')

In [102]:
diff_df

,0,result,res2
0,"Ужасно слабый аккумулятор, это основной минус ...",1,pos
1,ценанадежность-неубиваемостьдолго держит батар...,1,pos
2,"подробнее в комментариях К сожалению, факт по...",0,neg
3,я любительница громкой музыки. Тише телефона у...,0,neg
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",1,pos
5,- Удобная Клавиатура и русская раскладка - 2 с...,1,pos
6,Супер телефон! 1.QWERTY!!! Это самый лучший ег...,1,pos
7,- толщина (помещается даже в брюки) - аккумуля...,1,pos
8,Аккумулятор ужасен! Хватает буквально на неско...,0,neg
9,1 удобный.клавеатура просто класс быстро пишеш...,1,pos
